In [7]:
# Usei como base esse tutorial:
# https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/keras/regression.ipynb#scrollTo=f-OHX4DiXd8x


# Ja li tudo
# https://boostedml.com/2020/04/1-d-convolutional-neural-networks-for-time-series-basic-intuition.html

# How do I use make a convolutional layer for tabular (1-D) features?
# https://stackoverflow.com/questions/59756806/tensorflow-how-do-i-use-make-a-convolutional-layer-for-tabular-1-d-features

# TODO: Fiquei de verificar:
# https://machinelearningmastery.com/cnn-models-for-human-activity-recognition-time-series-classification/

<IPython.core.display.Javascript object>

In [8]:
%load_ext nb_black
%load_ext lab_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


<IPython.core.display.Javascript object>

# Lendo Dfs

In [9]:
import pathlib
import datetime

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
import datetime

from tensorflow import keras
from tensorflow.keras import layers

from keras.callbacks import ModelCheckpoint
import keras_tuner as kt

# 80%
PATH_TREINO = "../Data/3_Gold/2_Treino_all_stocks_80.csv"
DF_TREINO = pd.read_csv(PATH_TREINO, sep=",")
train_dataset = DF_TREINO
train_dataset = train_dataset.drop(
    [
        "oil_5",
        "usd_5",
        # "abev_5",
        # "jbs_5",
        "petr_5",
        "vale_5",
    ],
    axis=1,
)
train_labels = train_dataset.pop("ibova_5")

# 20%
PATH_TESTE = "../Data/3_Gold/2_Teste_all_stocks.csv"
DF_TESTE = pd.read_csv(PATH_TESTE, sep=",")
test_dataset = DF_TESTE
test_dataset = test_dataset.drop(
    [
        "oil_5",
        "usd_5",
        # "abev_5",
        # "jbs_5",
        "petr_5",
        "vale_5",
    ],
    axis=1,
)
test_labels = test_dataset.pop("ibova_5")

# 20%
PATH_VALIDACAO = "../Data/3_Gold/2_Validacao_all_stocks.csv"
DF_VALIDACAO = pd.read_csv(PATH_VALIDACAO, sep=",")
validacao_dataset = DF_VALIDACAO
validacao_dataset = validacao_dataset.drop(
    [
        "oil_5",
        "usd_5",
        # "abev_5",
        # "jbs_5",
        "petr_5",
        "vale_5",
    ],
    axis=1,
)
validacao_labels = validacao_dataset.pop("ibova_5")

"""
	Resultado
"""

list_activation = []
list_best_epoch = []
list_best_model_path = []
list_hp_unit_1 = []
list_hp_unit_2 = []
list_hp_unit_3 = []
list_hp_unit_4 = []
list_kernel_size_1 = []
list_kernel_size_2 = []
list_learning_rate = []
list_n_filter_1 = []
list_n_filter_2 = []
list_val_mae = []
list_val_mse = []


df_results = pd.DataFrame(
    columns=[
        "best_epoch",
        "val_mae",
        "val_mse",
        "n_filter_1",
        "n_filter_2",
        "kernel_size_1",
        "kernel_size_2",
        "learning_rate",
        "activation",
    ]
)

<IPython.core.display.Javascript object>

In [10]:
def get_results(history, best_hps, bst_model_path):
    # Select the best epoch,
    val_mse_per_epoch = history.history["val_mse"]
    best_epoch = val_mse_per_epoch.index(min(val_mse_per_epoch)) + 1

    try:
        value_activation = best_hps.get("activation")
    except:
        value_activation = None
    try:
        value_learning_rate = best_hps.get("learning_rate")
    except:
        value_learning_rate = None
    try:
        value_unit_1 = best_hps.get("unit_1")
    except:
        value_unit_1 = None
    try:
        value_unit_2 = best_hps.get("unit_2")
    except:
        value_unit_2 = None
    try:
        value_unit_3 = best_hps.get("unit_3")
    except:
        value_unit_3 = None
    try:
        value_unit_4 = best_hps.get("unit_4")
    except:
        value_unit_4 = None
    try:
        value_kernel_size_1 = best_hps.get("kernel_size_1")
    except:
        value_kernel_size_1 = None
    try:
        value_kernel_size_2 = best_hps.get("kernel_size_2")
    except:
        value_kernel_size_2 = None
    try:
        value_n_filter_1 = best_hps.get("n_filter_1")
    except:
        value_n_filter_1 = None
    try:
        value_n_filter_2 = best_hps.get("n_filter_2")
    except:
        value_n_filter_2 = None

    """"Repeticao MODEL.ipynb"""

    def save_in_list(one_list, value=None):
        try:
            one_list.append(value)
            return one_list
        except:
            one_list.append(None)
            return one_list

    save_in_list(list_activation, value_activation)
    save_in_list(list_best_epoch, best_epoch)
    save_in_list(list_best_model_path, bst_model_path)
    save_in_list(list_hp_unit_1, value_unit_1)
    save_in_list(list_hp_unit_2, value_unit_2)
    save_in_list(list_hp_unit_3, value_unit_3)
    save_in_list(list_hp_unit_4, value_unit_4)
    save_in_list(list_kernel_size_1, value_kernel_size_1)
    save_in_list(list_kernel_size_2, value_kernel_size_2)
    save_in_list(list_learning_rate, value_learning_rate)
    save_in_list(list_n_filter_1, value_n_filter_1)
    save_in_list(list_n_filter_2, value_n_filter_2)
    save_in_list(list_val_mae, min(history.history["val_mae"]))
    save_in_list(list_val_mse, min(history.history["val_mse"]))

    # print(f"list_activation: {list_activation}")
    # print(f"list_best_epoch: {list_best_epoch}")
    # print(f"list_best_model_path: {list_best_model_path}")
    # print(f"list_hp_unit_1: {list_hp_unit_1}")
    # print(f"list_hp_unit_2: {list_hp_unit_2}")
    # print(f"list_hp_unit_3: {list_hp_unit_3}")
    # print(f"list_hp_unit_4: {list_hp_unit_4}")
    # print(f"list_kernel_size_1: {list_kernel_size_1}")
    # print(f"list_kernel_size_2: {list_kernel_size_2}")
    # print(f"list_learning_rate: {list_learning_rate}")
    # print(f"list_n_filter_1: {list_n_filter_1}")
    # print(f"list_n_filter_2: {list_n_filter_2}")
    # print(f"list_val_mae: {list_val_mae}")
    # print(f"list_val_mse: {list_val_mse}")

<IPython.core.display.Javascript object>

In [11]:
def save_df_results(save_time):

    result_columns = [
        "activation",
        "best_epoch",
        "best_model_path",
        "hp_unit_1",
        "hp_unit_2",
        "hp_unit_3",
        "hp_unit_4",
        "kernel_size_1",
        "kernel_size_2",
        "learning_rate",
        "n_filter_1",
        "n_filter_2",
        "val_mae",
        "val_mse",
    ]

    data = list(
        zip(
            list_activation,
            list_best_epoch,
            list_best_model_path,
            list_hp_unit_1,
            list_hp_unit_2,
            list_hp_unit_3,
            list_hp_unit_4,
            list_kernel_size_1,
            list_kernel_size_2,
            list_learning_rate,
            list_n_filter_1,
            list_n_filter_2,
            list_val_mae,
            list_val_mse,
        )
    )

    df_resultados = pd.DataFrame(
        data,
        columns=result_columns,
    )

    df_resultados.to_csv(f"Resultados_{TYPE_MODEL}_{save_time}.csv")
    return df_resultados

<IPython.core.display.Javascript object>

# Definindo o modelo

In [12]:
def model_builder(hp):
    hp_unit_1 = int(len(train_dataset.keys()))
    hp_unit_2 = hp.Int("unit_2", min_value=4, max_value=64, step=1)
    hp_unit_3 = hp.Int("unit_3", min_value=4, max_value=64, step=1)
    hp_unit_4 = hp.Int("unit_4", min_value=4, max_value=64, step=1)
    activationL = [
        "sigmoid",
        "softplus",
        "softsign",
        "tanh",
        "selu",
        "elu",
        "relu",
    ]
    activation_choice = hp.Choice("activation", values=activationL)
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])

    model = keras.Sequential(
        [
            layers.Dense(
                units=hp_unit_1,
                activation=activation_choice,
                input_shape=[len(train_dataset.keys())],
            ),
            layers.Dense(units=hp_unit_2, activation=activation_choice),
            layers.Dense(units=hp_unit_3, activation=activation_choice),
            layers.Dense(units=hp_unit_4, activation=activation_choice),
            layers.Dense(units=1, activation="linear"),
        ]
    )

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss="mse",
        metrics=["mae", "mse"],
    )

    return model

<IPython.core.display.Javascript object>

# Selecionando a melhor epoca do modelo

In [13]:
def repetir(SAVE_TIME, EPOCHS, epochs_to_search, TYPE_MODEL):
    best_hps = None
    history = None
    bst_model_path = None
    tuner = None

    tuner = kt.RandomSearch(
        model_builder,
        objective="val_mse",
        # max_epochs=10,
        # factor=3,
        directory="logs",
        project_name=f"{TYPE_MODEL}_{SAVE_TIME}_hyper_parameters",
    )

    # Buscando hyper parametros
    tuner.search(
        train_dataset,
        train_labels,
        epochs=epochs_to_search,
        validation_split=0.2,
        verbose=0,
    )

    best_hps = tuner.get_best_hyperparameters(num_trials=100)[0]

    # Build the model with the optimal hyperparameters and train it on the data for 50 epochs
    bst_model_path = f"Models/{TYPE_MODEL}/model_{TYPE_MODEL}_{SAVE_TIME}.h5"
    model_checkpoint = ModelCheckpoint(
        bst_model_path, save_best_only=True, monitor="val_mse", mode="min"
    )

    model = tuner.hypermodel.build(best_hps)
    history = model.fit(
        train_dataset,
        train_labels,
        epochs=EPOCHS,
        validation_split=0.2,
        verbose=0,
        callbacks=([model_checkpoint]),
    )

    # Select the best epoch,
    val_mse_per_epoch = history.history["val_mse"]
    best_epoch = val_mse_per_epoch.index(min(val_mse_per_epoch)) + 1

    # Saving results in lists
    get_results(history, best_hps, bst_model_path)

<IPython.core.display.Javascript object>

In [14]:
TYPE_MODEL = "MLP"
REPETICAO = 3
EPOCHS = 500
epochs_to_search = 3

for i in range(REPETICAO):
    SAVE_TIME = datetime.datetime.strftime(datetime.datetime.now(), "%y_%m_%d_%Hh%Mm%S")
    repetir(SAVE_TIME, EPOCHS, epochs_to_search, TYPE_MODEL)

df_resultados = save_df_results(SAVE_TIME)
df_resultados

INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit


,activation,best_epoch,best_model_path,hp_unit_1,hp_unit_2,hp_unit_3,hp_unit_4,kernel_size_1,kernel_size_2,learning_rate,n_filter_1,n_filter_2,val_mae,val_mse
0,tanh,48,Models/MLP/model_MLP_22_12_14_20h07m00.h5,None,27,63,35,None,None,0.001,None,None,1.007162,1.632360
1,elu,134,Models/MLP/model_MLP_22_12_14_20h07m41.h5,None,43,19,20,None,None,0.010,None,None,1.004151,1.622221
2,tanh,344,Models/MLP/model_MLP_22_12_14_20h08m21.h5,None,43,38,16,None,None,0.010,None,None,0.999427,1.612047


<IPython.core.display.Javascript object>

In [15]:
# Just stop run jupyter
df_resultados.arthur briganti gini

SyntaxError: invalid syntax (975941084.py, line 2)

ERROR:root:Cannot parse: 2:21: df_resultados.arthur briganti gini
Traceback (most recent call last):
  File "c:\Users\arthu\.conda\envs\pystock\lib\site-packages\lab_black.py", line 218, in format_cell
    formatted_code = _format_code(cell)
  File "c:\Users\arthu\.conda\envs\pystock\lib\site-packages\lab_black.py", line 29, in _format_code
    return format_str(src_contents=code, mode=FileMode())
  File "c:\Users\arthu\.conda\envs\pystock\lib\site-packages\black\__init__.py", line 1163, in format_str
    dst_contents = _format_str_once(src_contents, mode=mode)
  File "c:\Users\arthu\.conda\envs\pystock\lib\site-packages\black\__init__.py", line 1173, in _format_str_once
    src_node = lib2to3_parse(src_contents.lstrip(), mode.target_versions)
  File "c:\Users\arthu\.conda\envs\pystock\lib\site-packages\black\parsing.py", line 128, in lib2to3_parse
    raise exc from None
black.parsing.InvalidInput: Cannot parse: 2:21: df_resultados.arthur briganti gini
ERROR:root:Cannot parse: 2:21: d

# Avaliando o modelo

Todas as métricas já foram extraidas, as análises posteriores são apenas para fins exploratorios 

In [ ]:
df_results.teste()

In [ ]:
print("Best epoch: %d" % (best_epoch,))
print(f'val_mae: {min(history.history["val_mae"])}')
print(f'val_mse: {min(history.history["val_mse"])}')

print(f'n_filter_1: {best_hps.get("n_filter_1")}')
# print(f'n_filter_2: {best_hps.get("n_filter_2")}')
print(f'kernel_size_1: {best_hps.get("kernel_size_1")}')
# print(f'kernel_size_2: {best_hps.get("kernel_size_2")}')

# print(f'unit_3: {best_hps.get("unit_3")}')
# print(f'unit_4: {best_hps.get("unit_4")}')
print(f'learning_rate: {best_hps.get("learning_rate")}')
print(f'activation: {best_hps.get("activation")}')

In [ ]:
# for REPETICAO, value in enumerate((df_results)):
df_results.loc[REPETICAO, "best_model_path"] = bst_model_path
df_results.loc[REPETICAO, "best_epoch"] = best_epoch
df_results.loc[REPETICAO, "val_mae"] = min(history.history["val_mae"])
df_results.loc[REPETICAO, "val_mse"] = min(history.history["val_mse"])
df_results.loc[REPETICAO, "n_filter_1"] = best_hps.get("n_filter_1")
df_results.loc[REPETICAO, "n_filter_2"] = None
df_results.loc[REPETICAO, "kernel_size_1"] = best_hps.get("kernel_size_1")
df_results.loc[REPETICAO, "kernel_size_2"] = None
df_results.loc[REPETICAO, "learning_rate"] = best_hps.get("learning_rate")
df_results.loc[REPETICAO, "activation"] = best_hps.get("activation")

df_results
# bst_model_path

In [ ]:
model.teste()

In [ ]:
df_results.to_csv(
    f"Parameter_{TYPE_MODEL}",
    header=True,
)

In [ ]:
model.summary()

# Treinando modelo

In [ ]:
# hypermodel = tuner.hypermodel.build(best_hps)
# history = hypermodel.fit(train_dataset, train_labels, epochs=best_epoch, validation_split=0.2, verbose=1)

# # ja pega o modelo na melhor epoca

In [ ]:
eval_result = model.evaluate(test_dataset, test_labels)
print("[test loss, test accuracy]:", eval_result)

# Avaliando o modelo

In [ ]:
# print(f'train_dataset: {train_dataset.columns()} \n')
# print(f'valid_dataset: {valid_dataset.columns()} \n')
# print(f'test_dataset: {test_dataset.columns()} \n')

In [ ]:
hist = pd.DataFrame(history.history)
hist["epoch"] = history.epoch
hist.tail()

In [ ]:
print(f"Min MSE: {hist.mse.min()}")
print(f"Min Val_MSE: {hist.val_mse.min()}")

In [ ]:
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist["epoch"] = history.epoch

    plt.figure()
    plt.xlabel("Epoch")
    plt.ylabel("Mean Abs Error [ibova_0]")
    plt.plot(hist["epoch"], hist["mae"], label="Train Error")
    plt.plot(hist["epoch"], hist["val_mae"], label="Val Error")
    plt.ylim([0, 1])
    plt.legend()

    plt.figure()
    plt.xlabel("Epoch")
    plt.ylabel("Mean Square Error [$ibova_0^2$]")
    plt.plot(hist["epoch"], hist["mse"], label="Train Error")
    plt.plot(hist["epoch"], hist["val_mse"], label="Val Error")
    plt.ylim([0, 1])
    plt.legend()
    plt.show()


plot_history(history)

# Realizando as previsões

In [ ]:
test_predictions = model.predict(test_dataset).flatten()

# Avaliando as previsões:

In [ ]:
def show_test_labels():
    # Imprimindo valores reais
    plt.plot(test_labels, color="r", label="ibova_REAL")
    plt.xlabel("Dates")
    plt.ylabel("Variation ROC")
    plt.title("ibova_REAL")
    plt.legend()


def show_model_predictions():
    # Imprimindo previsoes
    plt.plot(test_predictions, color="g", label="predictions_MODEL")
    plt.xlabel("Dates")
    plt.ylabel("Variation ROC")
    plt.title("predictions_MODEL")
    plt.legend()


def show_compare_graph():
    # Predictt X Real values
    plt.plot(test_labels, color="r", label="ibova_REAL")
    plt.plot(test_predictions, color="g", label="predictions_MODEL")
    plt.xlabel("Dates")
    plt.ylabel("Variation ROC")
    plt.title("Predict X Real values")
    plt.legend()
    plt.show()


def show_true_predict_values():
    plt.figure(figsize=(24, 4))
    plt.scatter(test_labels, test_predictions)
    plt.xlabel("True Values [ibova_0]")
    plt.ylabel("Predictions [ibova_0]")
    plt.axis("equal")
    plt.axis("square")
    plt.xlim([0, plt.xlim()[1]])
    plt.ylim([0, plt.ylim()[1]])
    _ = plt.plot([-100, 100], [-100, 100])

In [ ]:
show_compare_graph()

In [ ]:
show_model_predictions()

In [ ]:
model_name = f'mlp_unit_1({best_hps.get("unit_1")})unit_2({best_hps.get("unit_2")})unit_3({best_hps.get("unit_3")})unit_4({best_hps.get("unit_4")})_learning({best_hps.get("learning_rate")})_activation({best_hps.get("activation")})'
model_name
model.save(f"models/mlp/{model_name}date_{save_time}.h5")

In [ ]:
# Analisando Medias:

print(f"Massa de predição: {test_predictions.mean()}")
print(f"Massa inicial: {ibova_test.mean()}")
print(f"Diferenças das médias: {ibova_test.mean() - test_predictions.mean()}")

# Add o RM_MSE medio

In [ ]:
save_time